# 1. 与 google drive 链接，导入文件

In [25]:
from google.colab import drive
drive.mount('/content/drive/')

dir_path = '/content/drive/MyDrive/2023NLPCourse/Assignment1/Part_B'

%cd /content/drive/MyDrive/2023NLPCourse/Assignment1/Part_B

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/2023NLPCourse/Assignment1/Part_B


In [26]:
!pip install corextopic
!pip show scikit-learn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: fastai, imbalanced-learn, librosa, lightgbm, mlxtend, qudida, sklearn-pandas, yellowbrick


# 2. 处理数据以便输入 CorEx模型

In [ ]:
import numpy as np
import scipy.sparse as ss
import pickle
from corextopic import corextopic as ct
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from ast import literal_eval

data_anchored_corex = pd.read_excel("Data/preprocessed_data.xlsx")
train_topics = pd.read_excel("Data/Training_topics.xlsx")
df_anchored_corex = pd.DataFrame()

data_words_anchored_corex = []
for x in data_anchored_corex['Clean']:
    data_words_anchored_corex.append(' '.join(literal_eval(x)))
keywords = []
for x in train_topics['Keys']:
    keywords.append(literal_eval(x))

# 该模型传入的特征是 count Vector 词频向量
vectorizer_anchored_corex = CountVectorizer(stop_words='english', max_features=20000, binary=True)

doc_word_anchored_corex = vectorizer_anchored_corex.fit_transform(data_words_anchored_corex)
doc_word_anchored_corex = ss.csr_matrix(doc_word_anchored_corex)

words = list(np.asarray(vectorizer_anchored_corex.get_feature_names_out()))

# 3. 训练模型，总共得到6个主题

In [28]:
topic_model_anchored_corex = ct.Corex(n_hidden=6, words=words, max_iter=1000, verbose=False, seed=42)
topic_model_anchored_corex.fit(doc_word_anchored_corex, words=words, anchors = keywords, anchor_strength=3);

# 4. 保存模型

In [29]:
pickle.dump(topic_model_anchored_corex, open('Models/Anchored_CorEx_Train_model.model', 'wb'))
topic_list_anchored_corex = topic_model_anchored_corex.get_topics()

df_anchored_corex['Topics'] = topic_list_anchored_corex
df_anchored_corex.to_excel('Data/Anchored_CorEx_topics.xlsx')

# 5. 打印查看所有的主题

In [30]:
anchored_corex_topics = topic_model_anchored_corex.get_topics()
for n,topic in enumerate(anchored_corex_topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ', '.join(topic_words))

0: fit, large, small, little, big, medium, loose, normally, size, run
1: wash, shrink, dry, washing, fade, shrunk, water, cold, shrinkage, hold
2: fit, great, good, quality, nice, perfect, price, shirt, product, love
3: material, soft, fabric, durable, rough, snug, athletic, sew, comfortable, feel
4: color, look, long, sleeve, short, pocket, length, pretty, old, shape
5: buy, quality, purchase, value, cheap, money, poor, expensive, seller, high


# 6. 使用抽取好的主题为每个数据标记主题，结果保存在 CorEx_labelled_aspect.xlsx

In [31]:
import pandas as pd
import pickle
from ast import literal_eval

data = pd.read_excel("Data/preprocessed_data.xlsx")
aspect_list = [[],[],[],[],[],[]]
words_set = []
for x in data['Clean']:
    words_set.append(set(literal_eval(x)))

model = pickle.load(open("Models/Anchored_CorEx_Train_model.model", 'rb'))
topic_list = []
for i, topic_words in enumerate(model.get_topics()):
    topic_list.append(set([words[0] for words in topic_words if words[1] > 0]))

for words in words_set:
    for i,topic_words in enumerate(topic_list):
        if (words & topic_words):
            aspect_list[i].append(1)
        else :
            aspect_list[i].append(0)
for i in range(6):
    data['Topic ' + str(i)] = aspect_list[i]
data.to_excel('Data/CorEx_labelled_aspect.xlsx')